<a href="https://colab.research.google.com/github/shantanu2383/FAMA-FRENCH-5-FACTORS-ST-REVERSAL-LT-REVERSAL-MOMENTUM/blob/main/01_20_CONSTRUCTING_FF3_FACTORS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CONSTRUCT FF3 FROM CLEANED PRICE AND FUNDAMENTALS DATA AND COMPARE TO FAMA FRENCH DAILY 3 FACTORS

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf 
import numpy as np 
import matplotlib.pyplot as plt

!pip install pandasql
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA 
from sklearn.metrics import mean_squared_error, mean_absolute_error 

import math
import matplotlib.pyplot as plt 
from datetime import datetime
import seaborn as sns 
import pandasql as ps

from sqlite3 import connect
conn=connect(':memory:')

from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26787 sha256=18da601c1325f497cb5c710ecf5017b53b4703615bede13b067658fc03b4c3c4
  Stored in directory: /root/.cache/pip/wheels/ed/8f/46/a383923333728744f01ba24adbd8e364f2cb9470a8b8e5b9ff
Successfully built pandasql
Mounted at /content/gdrive


# IMPORT CLEANED DATAFRAME FROM DATA CLEANING NOTEBOOK WITH VALUE AND SIZE PORTFOL

In [ ]:
filepath="/content/gdrive/MyDrive/FIMA_2023_JANUARY/V2/DATALOAD/RAW_DATA/"

main_stocks_df=pd.read_csv(filepath + "pre sort: cleaned returns with size and value sorts")

In [ ]:
main_stocks_df.drop(columns=['Unnamed: 0', 'index', 'Unnamed: 0.1'], inplace=True)

In [ ]:
main_stocks_df

# ADD MARKET CAP AT JUNE TO CALCULATE WEIGHTED RETURNS

In [ ]:
main_stocks_df['date']=pd.to_datetime(main_stocks_df['date'], format="%Y-%m-%d")
stock_weights=main_stocks_df[main_stocks_df['date'].dt.month==6]

stock_weights.to_sql('stock_weights', conn, if_exists='replace')

query='''
        SELECT DISTINCT ticker, quarter, mkt_cap, reference_date
        FROM stock_weights
        '''

stock_weights=pd.read_sql(query, conn)

#merge these weights back into main stocks df
main_stocks_df['reference_date']=pd.to_datetime(main_stocks_df['reference_date'], format="%Y-%m-%d")

stock_weights['reference_date']=pd.to_datetime(stock_weights['reference_date'], format="%Y-%m-%d")

main_stocks_df.to_sql('main_stocks_df', conn, if_exists='replace')
stock_weights.to_sql('stock_weights', conn, if_exists='replace')


query='''
        SELECT DISTINCT main_stocks_df.*, stock_weights.mkt_cap as mkt_cap_weight
        FROM main_stocks_df
        LEFT JOIN stock_weights
        ON main_stocks_df.ticker=stock_weights.ticker AND main_stocks_df.reference_date=stock_weights.reference_date
        '''

main_stocks_df_weights=pd.read_sql(query, conn)

main_stocks_df=main_stocks_df_weights

In [ ]:
main_stocks_df.to_csv(filepath + "main_stocks_df_mktweights")

# PORTFOLIO CONSTRUCTION

In [ ]:
main_stocks_df['date']=pd.to_datetime(main_stocks_df['date'])

valueWeightRet=main_stocks_df.groupby(['date', 'size_portfolio', 'value_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['mkt_cap_weight'])))

#Create factors dataframe
main_stocks_df.to_sql("main_stocks_df", conn, if_exists='replace')

query=''' 
        SELECT DISTINCT date
        FROM main_stocks_df
        '''
factors=pd.read_sql(query, conn)
factors['date']=pd.to_datetime(factors['date'])

In [ ]:
#Add in the three factors (SMB, HML, MKT)

#Add SMB portfolios:
factors['smb']=""
for i in factors.index:
  dateofData=factors['date'][i]
  factors['smb'][i]=np.average(valueWeightRet[dateofData]["S"])-np.average(valueWeightRet[dateofData]["B"])
                                

<ipython-input-11-06eeb0b14857>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factors['smb'][i]=np.average(valueWeightRet[dateofData]["S"])-np.average(valueWeightRet[dateofData]["B"])


In [ ]:
#Change index of valueWeightedRet for HML

valueWeightRet=main_stocks_df.groupby(['date', 'value_portfolio', 'size_portfolio']).apply(lambda x: np.average(pd.to_numeric(x['ret']), weights=pd.to_numeric(x['mkt_cap_weight'])))

#Add HML portfolios
factors['hml']=""

for i in factors.index:
  dateofData=factors['date'][i]
  factors['hml'][i]=np.average(valueWeightRet[dateofData]["H"])-np.average(valueWeightRet[dateofData]["L"])


In [ ]:
#Add mkt (monthly average returns) across all stocks

factors['mkt']=""
main_stocks_df.dropna(inplace=True)
weightedAvgMonthlyRet=main_stocks_df.groupby('date').apply(lambda x: np.average(pd.to_numeric(x.ret), weights=pd.to_numeric(x.mkt_cap_weight)))

for i in factors.index:
  dateofData=factors['date'][i]
  factors['mkt'][i]=weightedAvgMonthlyRet[dateofData]

In [ ]:
main_stocks_df=main_stocks_df.fillna(0)
#Add mkt ( daily returns) across all stocks
factors['mkt']=""
main_stocks_df.dropna(inplace=True)
weighted_dailyRet=main_stocks_df.groupby('date').apply(lambda x: np.average(pd.to_numeric(x.ret), weights=pd.to_numeric(x.mkt_cap_weight)))

for i in factors.index:
  dateofData=factors['date'][i]
  factors['mkt'][i]=weighted_dailyRet[dateofData]

<ipython-input-14-8fb2179436e5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  factors['mkt'][i]=weighted_dailyRet[dateofData]


In [ ]:
factors

# READ IN KENNETH FRENCH DATA FOR COMPARISON

In [ ]:
import pandas_datareader.data as web
from pandas_datareader.famafrench import get_available_datasets
datasets = get_available_datasets()
df_3_factor=[dataset for dataset in datasets if 'Research' in dataset and 'Factor' in dataset]

In [ ]:
df_3_factor
ff=web.DataReader(df_3_factor[2],'famafrench',start='2018-07-01',end='2022-11-01')[0]
ff['dd']=ff.index
ff['dd']=pd.to_datetime(ff['dd'], format="%Y-%m-%d")
factors['date']=pd.to_datetime(factors['date'])
ff=ff[ff['dd']>="2019-06-03"]

In [ ]:
ff=ff.rename(columns={'Mkt-RF': 'MKT'})

In [ ]:
ff.to_sql('ff', conn, if_exists='replace')

query='''
        SELECT DISTINCT dd as date, SMB as SMB_ff, HML as HML_ff, MKT as MKT_ff
        FROM FF
        '''
ff=pd.read_sql(query, conn)

In [ ]:
ff['date']=pd.to_datetime(ff['date'], errors='coerce')

x=["SMB_ff", "HML_ff"]
for x in x:
  ff[x]=pd.to_numeric(ff[x], errors='coerce')

y=['smb', 'hml']

for y in y:
  factors[y]=pd.to_numeric(factors[y], errors='coerce')

In [ ]:
ff

In [ ]:
factors.to_sql('factors', conn, if_exists='replace')
ff.to_sql('ff', conn, if_exists='replace')

query='''
        SELECT factors.date, factors.smb, factors.hml, factors.mkt, FF.SMB_ff, FF.HML_ff, FF.MKT_ff
        FROM factors
        LEFT JOIN ff
        ON factors.date=ff.date
        '''

ff_checker=pd.read_sql(query, conn)

In [ ]:
ff_checker

,date,smb,hml,mkt,SMB_ff,HML_ff,MKT_ff
0,2019-06-03 00:00:00,0.468934,2.311182,-0.773308,0.35,1.65,-0.40
1,2019-06-04 00:00:00,0.652171,-0.201773,2.276816,0.42,-0.02,2.33
2,2019-06-05 00:00:00,-0.895664,-1.247382,0.608100,-0.98,-0.97,0.70
3,2019-06-06 00:00:00,-1.227373,0.155641,0.574624,-1.04,0.01,0.55
4,2019-06-07 00:00:00,-0.005720,-0.880596,1.447050,0.05,-1.20,1.04
...,...,...,...,...,...,...,...
897,2022-12-21 00:00:00,0.034963,0.043678,1.374257,NaN,NaN,NaN
898,2022-12-22 00:00:00,0.225827,0.499582,-1.784820,NaN,NaN,NaN
899,2022-12-23 00:00:00,0.155313,2.584804,0.498839,NaN,NaN,NaN
900,2022-12-27 00:00:00,-0.813249,1.879353,-1.054648,NaN,NaN,NaN


In [ ]:
ff_checker.corr(method='pearson')

,smb,hml,mkt,SMB_ff,HML_ff,MKT_ff
smb,1.000000,0.059217,0.288949,0.914253,0.071141,0.296438
hml,0.059217,1.000000,-0.117748,0.072170,0.894678,-0.053778
mkt,0.288949,-0.117748,1.000000,0.194228,-0.070332,0.991765
SMB_ff,0.914253,0.072170,0.194228,1.000000,0.015418,0.194777
HML_ff,0.071141,0.894678,-0.070332,0.015418,1.000000,-0.016280
MKT_ff,0.296438,-0.053778,0.991765,0.194777,-0.016280,1.000000
